In this notebook, we will use SVM(support vector machine) to built and train a model using a human cell rocords, and classify the cells whether the samples belongs to a class are benign and malignant

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* Import required libraries**

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Load  and understand the dataset
Cell_data = pd.read_csv("../input/cell-sample/cell_samples.csv")
Cell_data.head()

In [ ]:
Cell_data.dtypes

In [ ]:
#find the shape of dataset
Cell_data.shape

In [ ]:
#find the some bottom data from datast
Cell_data.tail()

In [ ]:
#find data field information
Cell_data.info()

In [ ]:
#DEscribe the dataset-Statistical view of the dataset
Cell_data.describe()

In [ ]:
#Drop unnecessary features
Cell_data.drop('ID',axis=1,inplace=True)
Cell_data

In [ ]:
Cell_data = Cell_data[pd.to_numeric(Cell_data['BareNuc'], errors='coerce').notnull()]
Cell_data['BareNuc'] = Cell_data['BareNuc'].astype('int')
Cell_data.dtypes

# > ******Visualization the dataset****

In [ ]:
ax = Cell_data[Cell_data['Class'] == 4][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='red', label='malignant');
Cell_data[Cell_data['Class'] == 2][0:50].plot(kind='scatter', x='Clump', y='UnifSize', color='green', label='benign', ax=ax);
plt.show()

In [ ]:
#distribution of each columns of a dataset
sns.pairplot(Cell_data)

In [ ]:
sns.heatmap(Cell_data.corr(),annot=True)

In [ ]:
sns.distplot(Cell_data['UnifSize'])

# Handling missing values

In [ ]:
#Check the null/missing value in dataset
Cell_data.isnull().sum()

In [ ]:
#visualize null value through garphs
sns.heatmap(Cell_data.isnull(),yticklabels=True,cmap='viridis',annot=True)

# Split the dataset

In [ ]:
feature_df = Cell_data[['Clump', 'UnifSize', 'UnifShape', 'MargAdh', 'SingEpiSize', 'BareNuc', 'BlandChrom', 'NormNucl', 'Mit']]
X = np.asarray(feature_df)
X[0:5]

In [ ]:
Cell_data['Class'] = Cell_data['Class'].astype('int')
y = np.asarray(Cell_data['Class'])
y [0:5]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

# Build a prediction model

In [ ]:
#svm is used for both regession and classification.svm machine used to find a hyperplane in a N dimensional space
#Hyperplane-It is a decision boundaries that help classify the data points.
#Kernel-It is a function uesd to map a lower dimensional data into higher dimensional data
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

# Make prediction

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
y_pred.shape

# Find out the accuracy of model

In [ ]:
#R-squared error-it is the statistical measure thet represents the goodness of a fit of a regression line.
#The ideal value of r-Squared error is 1
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score,mean_absolute_error
r2_score(y_test,y_pred)

In [ ]:
#it is the number of correct prediction made as a ratio of all the prediction made.
accuracy_score(y_test,y_pred)

In [ ]:
#The average of square of the difference b/w original values and predicted values.
mean_squared_error(y_test,y_pred)

In [ ]:
# it is the average of the absolute differences b/w predictions and actual values 
mean_absolute_error(y_test,y_pred)

In [ ]:
#Confusion matrix-
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred, labels=[2,4])
np.set_printoptions(precision=2)

print (classification_report(y_test, y_pred))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Benign(2)','Malignant(4)'],normalize= False,  title='Confusion matrix')

# The End